In [1]:
%load_ext autoreload
%autoreload 2
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree
import datetime
import json

In [2]:
import requests

url = "https://www.sitimapa.com.co/server_side/georreferenciadorDirecciones_dian.php"
data = {"direccion": "carrera 57 22a 41 bogota"}

response = requests.post(url, data=data)

print(response.status_code)
print(response.text)

200
{"data":{"dirtrad":"CR 57 22 A 41 BOGOTA","validacion":"99","validacionPlaca":"99","fuente":"NUEVA","dirAlterna":"NO CAMBIO","esambigua":"","barrioTraducido":"","coddireccion":"0000000000KR0570220A0000000041000000000000000000000000000000000000000000000000","coddirplaca":"0000000000KR0570220A0000000041","nivsocio":"4","barrio":"CJR PARQUE DE CADIZ","localidad":"TEUSAQUILLO","longitude":"-74.10259114","latitude":"4.64089395","zonapostal":"111321229","coddane":"11001000","estado":"M"},"message":"Geocodificacion Exitosa.","success":true,"total_georreferenciadas_hoy":10,"code":"200","geo_limite":20,"descripcion":"Aun queda un total de 10 direcciones disponibles para el dia de hoy"}


In [32]:
data = pd.read_csv('final_data_async.csv', index_col=0)

In [33]:
json_data = data.to_json(orient='records')


In [37]:
9json_data

'[{"carro":"Renault Duster 1.6 Dynamique Gt 4x2","price":47500000,"link":"https:\\/\\/carro.mercadolibre.com.co\\/MCO-2151389978-renault-duster-16-dynamique-gt-_JM#position=1&search_layout=grid&type=item&tracking_id=a954cc2a-099c-416b-821a-cd14c0f7e95b","carro_info":"{\'Year\': \'2015\', \'Kilometrage\': \'75300 \', \'Publication Date\': \'Publicado hace 1 d\\u00eda\', \'Location\': \'La Colina Campestre - Suba - Bogot\\u00e1 D.C.\', \'Pub Number\': \'2151389978\', \'Created At\': \'2024-01-14 14:57:14\'}"},{"carro":"Bmw X1 2.0 Sdrive 18d","price":96000000,"link":"https:\\/\\/carro.mercadolibre.com.co\\/MCO-2151412446-bmw-x1-20-sdrive-18d-_JM#position=2&search_layout=grid&type=item&tracking_id=a954cc2a-099c-416b-821a-cd14c0f7e95b","carro_info":"{\'Year\': \'2017\', \'Kilometrage\': \'88523 \', \'Publication Date\': \'Publicado hace 1 d\\u00eda\', \'Location\': \'Santa B\\u00e1rbara II - Suba - Bogot\\u00e1 D.C.\', \'Pub Number\': \'2151412446\', \'Created At\': \'2024-01-14 14:57:15\'}

In [41]:
out = json.loads(json_data)

In [44]:
out

[{'carro': 'Renault Duster 1.6 Dynamique Gt 4x2',
  'price': 47500000,
  'link': 'https://carro.mercadolibre.com.co/MCO-2151389978-renault-duster-16-dynamique-gt-_JM#position=1&search_layout=grid&type=item&tracking_id=a954cc2a-099c-416b-821a-cd14c0f7e95b',
  'carro_info': "{'Year': '2015', 'Kilometrage': '75300 ', 'Publication Date': 'Publicado hace 1 día', 'Location': 'La Colina Campestre - Suba - Bogotá D.C.', 'Pub Number': '2151389978', 'Created At': '2024-01-14 14:57:14'}"},
 {'carro': 'Bmw X1 2.0 Sdrive 18d',
  'price': 96000000,
  'link': 'https://carro.mercadolibre.com.co/MCO-2151412446-bmw-x1-20-sdrive-18d-_JM#position=2&search_layout=grid&type=item&tracking_id=a954cc2a-099c-416b-821a-cd14c0f7e95b',
  'carro_info': "{'Year': '2017', 'Kilometrage': '88523 ', 'Publication Date': 'Publicado hace 1 día', 'Location': 'Santa Bárbara II - Suba - Bogotá D.C.', 'Pub Number': '2151412446', 'Created At': '2024-01-14 14:57:15'}"},
 {'carro': 'Volkswagen Gol 1.6 Ls',
  'price': 31000000,
  

In [43]:
json.dumps(out)

'[{"carro": "Renault Duster 1.6 Dynamique Gt 4x2", "price": 47500000, "link": "https://carro.mercadolibre.com.co/MCO-2151389978-renault-duster-16-dynamique-gt-_JM#position=1&search_layout=grid&type=item&tracking_id=a954cc2a-099c-416b-821a-cd14c0f7e95b", "carro_info": "{\'Year\': \'2015\', \'Kilometrage\': \'75300 \', \'Publication Date\': \'Publicado hace 1 d\\u00eda\', \'Location\': \'La Colina Campestre - Suba - Bogot\\u00e1 D.C.\', \'Pub Number\': \'2151389978\', \'Created At\': \'2024-01-14 14:57:14\'}"}, {"carro": "Bmw X1 2.0 Sdrive 18d", "price": 96000000, "link": "https://carro.mercadolibre.com.co/MCO-2151412446-bmw-x1-20-sdrive-18d-_JM#position=2&search_layout=grid&type=item&tracking_id=a954cc2a-099c-416b-821a-cd14c0f7e95b", "carro_info": "{\'Year\': \'2017\', \'Kilometrage\': \'88523 \', \'Publication Date\': \'Publicado hace 1 d\\u00eda\', \'Location\': \'Santa B\\u00e1rbara II - Suba - Bogot\\u00e1 D.C.\', \'Pub Number\': \'2151412446\', \'Created At\': \'2024-01-14 14:57:15

In [36]:
def get_soup_by_url(url, product: str = None):
    if product is None:
        url = url
    else:
        url = f'{url}/{product}'
    r = requests.get(url=url)
    s = BeautifulSoup(r.content, 'html.parser')
    return s

def get_vehicle_info(url):
    s = get_soup_by_url(url)
    text = s.find_all('span', attrs= {'class':'ui-pdp-subtitle'})[0].text.replace('.', '')
    parts = text.split(' · ')
    location = s.find('p', attrs = {'class':'ui-seller-info__status-info__subtitle'}).text
    pub_number = s.find_all('span', attrs= {'class':'ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD'})[0].text.replace('#','')
    year = parts[0].split(' | ')[0]
    kilometrage = parts[0].split(' | ')[1].replace('km', '')
    publication_date = parts[1]
    output_dict = {
        "Year": year,
        "Kilometrage": kilometrage,
        "Publication Date": publication_date,
        "Location": location,
        "Pub Number": pub_number,
        "Created At": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    return output_dict
data = pd.read_csv('final_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'final_data.csv'

In [21]:
data['link'][:5].apply(get_vehicle_info)

0    {'Year': '2017', 'Kilometrage': '47800 ', 'Pub...
1    {'Year': '2015', 'Kilometrage': '72000 ', 'Pub...
2    {'Year': '2015', 'Kilometrage': '73251 ', 'Pub...
3    {'Year': '2012', 'Kilometrage': '75500 ', 'Pub...
4    {'Year': '2017', 'Kilometrage': '80000 ', 'Pub...
Name: link, dtype: object

In [19]:
data['link'].apply(get_vehicle_info)

KeyboardInterrupt: 

In [2]:
link = 'https://carro.mercadolibre.com.co/MCO-1379439831-mercedes-benz-gle-350-d-4matic-2017-_JM#position=1&search_layout=grid&type=item&tracking_id=79a64cb3-adcf-4d29-be31-812b12b34cef'

In [11]:
link

'https://carro.mercadolibre.com.co/MCO-1379439831-mercedes-benz-gle-350-d-4matic-2017-_JM#position=1&search_layout=grid&type=item&tracking_id=79a64cb3-adcf-4d29-be31-812b12b34cef'

In [3]:
r = requests.get(url=link)

s = BeautifulSoup(r.content, 'html.parser')

In [4]:
# publication number

int(s.find_all('span', attrs= {'class':'andes-money-amount__fraction'})[0].text.replace('.', ''))

189000000

In [5]:
# precio
s.find_all('span', attrs= {'class':'ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD'})[0].text.replace('#','')

'1379439831'

In [16]:
import datetime

text = s.find_all('span', attrs= {'class':'ui-pdp-subtitle'})[0].text.replace('.', '')
parts = text.split(' · ')
location = s.find('p', attrs = {'class':'ui-seller-info__status-info__subtitle'}).text
pub_number = s.find_all('span', attrs= {'class':'ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD'})[0].text.replace('#','')
year = parts[0].split(' | ')[0]
kilometrage = parts[0].split(' | ')[1].replace('km', '')
publication_date = parts[1]
output_dict = {
  "Year": year,
  "Kilometrage": kilometrage,
  "Publication Date": publication_date,
  "Location": location,
  "Pub Number": pub_number,
  "Created At": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

output_dict

{'Year': '2017',
 'Kilometrage': '47800 ',
 'Publication Date': 'Publicado hace 3 días',
 'Location': 'Chicó Navarra - Usaquén - Bogotá D.C.',
 'Pub Number': '1379439831',
 'Created At': '2024-01-13 13:35:22'}

'Chicó Navarra - Usaquén - Bogotá D.C.'

In [8]:
table = s.find_all('table', attrs= {'class': 'andes-table'})

In [9]:
table

[]

In [10]:
rows = table[0].find_all('tr')
rows

IndexError: list index out of range

In [106]:
data = {}
for row in rows:
  cols = row.find_all(['th', 'td'])
  cols = [ele.text.strip() for ele in cols]
  data[cols[0]] = cols[1]


In [107]:
data

{'Marca': 'Mercedes-Benz',
 'Modelo': 'Clase GLE',
 'Año': '2017',
 'Versión': '3.0 Coupe 4matic Diésel',
 'Color': 'Gris',
 'Tipo de combustible': 'Diésel',
 'Puertas': '5',
 'Transmisión': 'Automática',
 'Motor': '3.0',
 'Tipo de carrocería': 'Station Wagon',
 'Kilómetros': '47800 km'}

In [22]:
rows

NameError: name 'rows' is not defined

In [49]:
dom = etree.HTML(str(s))

In [50]:
dom

<Element html at 0x7baa0c6bbfc0>

In [51]:
dom.xpath("//table[@class = 'andes-table']")

[]